In [1]:
import os
import json
from bson import json_util
from typing import List, Dict, TypedDict, Annotated, Sequence, Optional
# from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from pymongo import MongoClient
from operator import add
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
MONGO_URI = os.getenv("MONGODB_URI")
DB_NAME = os.getenv("MONGODB_DB_NAME")
collection_name = 'si'
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "containergenie.ai"

llm = ChatOpenAI(model='gpt-4o-mini',temperature=0.0)

def fetch_data_from_mongodb(collection_name: str, query: Dict = None, limit: int = None) -> List[Dict]:
 
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]
    collection = db[collection_name]
    
    # Prepare the find operation
    find_operation = collection.find(query) if query else collection.find()
    
    # Fetch and return the data
    data = list(find_operation)
    
    # Close the connection
    client.close()
    
    return data

In [9]:
booking_reference="CHERRY202409072244"

In [12]:
si_data = fetch_data_from_mongodb(collection_name=collection_name, query={"bookingReference": booking_reference})

In [17]:
si_data

[{'_id': ObjectId('66e1562f0abd52a6a79a2ff6'),
  'bookingReference': 'CHERRY202409072244',
  'voyageDetails': {'vesselName': 'APL TEMASEK', 'voyageNumber': '2024581E'},
  'routeDetails': {'placeOfReceipt': 'NINGBO',
   'portOfLoading': 'NINGBO',
   'portOfDischarge': 'YANTIAN',
   'placeOfDelivery': 'YANTIAN'},
  'paymentDetails': {'freightPaymentTerms': 'COLLECT',
   'freightPayableAt': 'ROTTERDAM, NETHERLANDS'},
  'documentationDetails': {'blType': 'NEGOTIABLE',
   'numberOfOriginalBLs': 3,
   'numberOfCopies': 0},
  'partyDetails': {'shipper': {'name': 'SHIPPER 5029',
    'address': 'NO. 188, SHANXI ROAD, NINGBO, CHINA',
    'telephone': '+86 574 8765 4321',
    'email': 'INFO@CHINAIMPORTEXPORT.COM'},
   'consignee': {'name': 'EUROTECH TRADING BV',
    'address': 'WATERLOO PLAZA, 121-123 WATERLOOPLEIN, 1011 PG AMSTERDAM, NETHERLANDS',
    'taxId': 'NL856321452B01',
    'president': 'MR. JAN VAN DER WIEL',
    'telephone': '+31 20 624 3500',
    'fax': '+31 20 624 3501'},
   'notifyP

In [26]:
import asyncio
from verify_policy import RAGModel

def main():
    # Language model과 프롬프트 템플릿 설정
    template = "Given the following SI data: {si_data}, provide relevant document updates."
    llm = ChatOpenAI(temperature=0, 
                    model_name="gpt-4o-mini",
                    streaming=True,              
                    callbacks=[StreamingStdOutCallbackHandler()]
                    )
    
    # RAG 모델 초기화
    rag_model = RAGModel(llm, ['./si_validation_story/resources/docs/cherry_compliance.pdf'], template)
    
    # SI 데이터를 입력하여 응답 생성
    response = rag_model.invoke(si_data)
    return response

# 실행
res = main()

Loading existing vector store from 'faiss_index'.


In [27]:
res

<coroutine object RAGModel.invoke at 0x11fc6bc40>

In [20]:
import asyncio
import aiohttp
from web_search import WebSearch  # WebSearch 모듈이 같은 디렉토리에 있어야 합니다

async def main():
    async with aiohttp.ClientSession() as session:
        # WebSearch 클래스 인스턴스 생성
        web_search = WebSearch(session)
        
        # 포함할 URL과 제외할 URL 설정 (옵션)
        web_search.set_include_urls(['google.com'])
        web_search.set_exclude_urls(['exclude.com'])
        
        # 검색 수행
        query = "AI technologies in healthcare"
        results = await web_search.search(query)
        
        print(f"Search results for '{query}':")
        for result in results:
            print(result)

# Jupyter Notebook에서 비동기 함수 실행
await main()

Performing web search for query: AI technologies in healthcare
Failed to fetch results from Tavily. Status: 405
Search results for 'AI technologies in healthcare':
